## Environment Setup

In [515]:
# Set up the environment
%load_ext rpy2.ipython
import rpy2.robjects as robjects
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import LDA, LDAModel
import numpy as np
import scipy as sp
import time
from scipy import optimize
import os
import pandas as pd
from scipy import io
from rpy2.robjects import pandas2ri
import rpy2

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## R Imports for STM

In [4]:
for f in os.listdir("R"):
    if f not in ['.DS_Store', '.Rapp.history', 'box', 'e_step_spark.R']:
        robjects.r.source("R/" + f)

In [5]:
robjects.r('''
    library(Matrix); library(stringr); library(splines)
''')

array(['splines', 'stringr', 'Matrix', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], 
      dtype='|S9')

## Sample run with R code

In [6]:
# Subsample the corpus to only consider articles mentioning 'monte carlo'
cond_mat_mc = pd.read_csv('cond_mat_mc.csv')
%Rpush cond_mat_mc

In [7]:
# Prep the corpus
robjects.r('''
    processed_corpus_temp = textProcessor(cond_mat_mc$abstract, metadata=cond_mat_mc, lowercase=TRUE)
    processed_corpus = prepDocuments(processed_corpus_temp$documents,
                                 processed_corpus_temp$vocab, 
                                 processed_corpus_temp$meta,
                                 lower.thresh=1)
    rm(processed_corpus_temp); invisible(gc())
''');

Building corpus... 
Converting to Lower Case... 
Removing stopwords... 
Removing numbers... 
Removing punctuation... 
Stemming... 
Creating Output... 
Removing 8267 of 15095 terms (8267 of 363331 tokens) due to frequency 
Your corpus now has 6359 documents, 6828 terms and 355064 tokens.

In [9]:
robjects.r('''
    fit = stm(processed_corpus$documents, 
             processed_corpus$vocab,
             K=20,
             data=processed_corpus$meta,
             init.type = 'Spectral',
             seed=02138)
''');

In [10]:
robjects.r('''
    documents <- fit$documents
    vocab <- fit$vocab
    settings <- fit$settings 
    model <- fit$model
    verbose <- settings$verbose
  ##########
  #Step 1: Initialize Parameters
  ##########
    ngroups <- settings$ngroups
  if(is.null(model)) {
    if(verbose) cat("Beginning Initialization.\n")
    #initialize
    model <- stm.init(documents, settings)
    #if we were using the Lee and Mimno method of setting K, update the settings
    if(settings$dim$K==0) settings$dim$K <- nrow(model$beta[[1]])
    #unpack
    mu <- list(mu=model$mu)
    sigma <- model$sigma
    beta <- list(beta=model$beta)
    if(!is.null(model$kappa)) beta$kappa <- model$kappa
    lambda <- model$lambda
    convergence <- NULL 
    #discard the old object
    rm(model)
  } else {
    if(verbose) cat("Restarting Model...\n")
    #extract from a standard STM object so we can simply continue.
    mu <- model$mu
    beta <- list(beta=lapply(model$beta$logbeta, exp))
    if(!is.null(model$beta$kappa)) beta$kappa <- model$beta$kappa
    sigma <- model$sigma
    lambda <- model$eta
    convergence <- model$convergence
    #manually declare the model not converged or it will stop after the first iteration
    convergence$stopits <- FALSE
    convergence$converged <- FALSE
    #iterate by 1 as that would have happened otherwise
    convergence$its <- convergence$its + 1 
  }    
  
  #Pull out some book keeping elements
  ntokens <- sum(settings$dim$wcounts$x)
  betaindex <- settings$covariates$betaindex
  stopits <- FALSE
  if(ngroups!=1) {
    groups <- cut(1:length(documents), breaks=ngroups, labels=FALSE) 
  }
  suffstats <- vector(mode="list", length=ngroups)
''');

Beginning Initialization.
	 Calculating the gram matrix...
	 Finding anchor words...
 	....................
	 Recovering initialization...
 	....................................................................
Initialization complete.


In [15]:
def rlist_2py(rlist):
    return dict(zip(rlist.names,
               list(rlist)))

In [16]:
fit = dict(zip( robjects.globalenv['fit'].names, 
         list( robjects.globalenv['fit'])))
settings = dict(zip( fit['settings'].names, 
         list(fit['settings'])))

In [17]:
K, A, V, N = [int(settings['dim'][i][0]) for i in range(4)]

In [514]:
# Some setup for EM, retrieving the R objects
stopits = False
ngroups = int(robjects.globalenv['ngroups'][0])
documents = [np.array(x) for x in list(robjects.globalenv['documents'])]
beta_index = np.array(robjects.globalenv['betaindex'])
beta = [np.array(x) for x in robjects.globalenv['beta'][0]]
update_mu = True
Lambda = np.array(robjects.globalenv['lambda'])
mu = np.array(robjects.globalenv['mu'][0])
sigma = np.array(robjects.globalenv['sigma'])
verbose = settings['verbose'][0]

# Run EM
for i in range(2):
    
    # Non-blocked update
    if ngroups==1:
        # TODO
        a = 0

In [541]:
# Run all the necessary imports
def run_imports(x):
    import scipy as sp
    import numpy as np
    from scipy import optimize
    return 1

outs = sc.parallelize(range(100)).map(run_imports).collect()

In [509]:
def likelihood(eta, beta, doc_ct, mu, siginv):
    exp_eta = np.exp(np.append(eta, np.array([0])))
    ndoc = np.sum(doc_ct)
    part1 = np.dot(np.log(np.dot(exp_eta, beta)), doc_ct) - ndoc * np.log(np.sum(exp_eta))
    diff = mu.T - eta
    part2 = 0.5 * float(np.dot(np.dot(diff, siginv), diff.T))
    return part2 - part1

In [510]:
def grad(eta, beta, doc_ct, mu, siginv):
    exp_eta = np.exp(np.append(eta, [0]))
    beta_prime = np.apply_along_axis(lambda x: x * exp_eta, 0, beta)
    part1 = np.dot(beta_prime, doc_ct/np.sum(beta_prime, 0).T) - (np.sum(doc_ct)/ np.sum(exp_eta)) * exp_eta
    diff = mu.T - eta
    part2 = np.dot(siginv, diff.T)
    part1 = part1[:len(part1)-1]
    return (part2.T - part1).flatten()

In [533]:
def estep_docloop(doc_item, siginv, sigmaentropy):
    doc_ct = doc_item['doc'][1]
    eta = doc_item['init']
    beta = doc_item['beta_i']
    mu = doc_item['mu_i']
    optim_par = sp.optimize.minimize(likelihood, eta, args=(beta, doc_ct, mu, siginv), 
                            method='BFGS')
    
    def hpb(eta, beta, doc_ct, mu, siginv, sigmaentropy):
        
        # Compute the Hessian
        exp_eta = np.exp(np.append(eta, [0]))
        theta = np.reshape(exp_eta/np.sum(exp_eta), (len(exp_eta), -1)).T
        EB = np.apply_along_axis(lambda x: x * exp_eta, 0, beta)
        EB = np.apply_along_axis(lambda x: x * (np.sqrt(doc_ct).T) / np.sum(EB,0), 1, EB)
        hess = np.dot(EB, EB.T) - np.sum(doc_ct) * np.dot(theta.T, theta)    
        EB = np.apply_along_axis(lambda x: x * np.sqrt(doc_ct).T, 1, EB)
        hess[np.diag_indices_from(hess)] = hess[np.diag_indices_from(hess)] - np.sum(EB, 1) + np.sum(doc_ct) * theta
        hess = hess[:hess.shape[0]-1,:hess.shape[1]-1] + siginv

        # Invert via Cholesky decomposition
        try:
            nu = np.linalg.cholesky(hess)
        except:
            dvec = np.array(np.diag(hess))
            magnitudes = np.sum(np.abs(hess), 1) - abs(dvec)
            Km1 = len(dvec)
            for i in range(Km1):
                if dvec[i] < magnitudes[i]:
                    dvec[i] = magnitudes[i]
            hess[np.diag_indices_from(hess)] = dvec
            nu = np.linalg.cholesky(hess)

        # Finish construction
        det_term = -np.sum(np.log(np.diag(nu)))
        nu = np.linalg.inv(np.triu(nu))
        nu = np.dot(nu, nu.T)
        diff = eta - mu.flatten()

        # Compute the bound
        bound = (np.dot(np.log(np.dot(theta, beta)), doc_ct) + det_term 
                 - 0.5 * np.dot(diff.T, np.dot(siginv, diff)) - sigmaentropy)

        # Construct ouput
        out = {'phis': EB,
               'eta': {'lambda': eta, 'nu': nu},
               'bound': bound}
        return out
    
    return hpb(optim_par.x, beta, doc_ct, mu, siginv, sigmaentropy)

In [517]:
def estep_spark(documents, beta_index, beta, Lambda_old,
                mu, sigma, verbose, sc, update_mu=False):
    
    # Initialize sufficient statistics
    sigma_ss = np.zeros((K-1, K-1))
    beta_ss = [np.zeros((K, V)) for i in range(A)]
    bound = np.array([0] * N)
    Lambda = np.array([0] * N)
    siginv = np.linalg.inv(sigma)
    sigmaentropy = np.log(np.abs(np.linalg.det(sigma))) * 0.5
    
    # Parallelize document collection
    collection = [{'doc':doc, 'aspect': int(aspect), 'init': init} 
                  for (doc, aspect, init) in zip(documents, beta_index, Lambda_old)]
    for item in collection:
        item['beta_i'] = beta[item['aspect']-1][:,[x-1 for x in item['doc'][0]]]
        item['mu_i'] = mu
    collection_par = sc.parallelize(collection)
    
    return collection

In [ ]:
# These operations wil eventually be run within estep_spark; just here for speed's sake
collection = estep_spark(documents, beta_index, beta, Lambda, mu, sigma, verbose, sc, False)
collection_par = sc.parallelize(collection[:1])
collection_par.collect()
trial_run = collection_par.map(lambda x: estep_docloop(x, siginv, sigmaentropy)).collect()
print trial_run

## STM Class Definitions

In [ ]:
import pandas.rpy.common as rcom
iris = rcom.load_data('iris')
iris.head()

In [ ]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry

class STM(object):
    
    def __init__(self, sc):
        self.sc = sc
        self.status = 0
        self.theta = None
        self.n_partitions = None # Get this from Spark context
        self.lhood_bound = None
        self.seed = None
        self.description = None
    
    def __e_step__(self, documents):
        mu_i = self.mu
        sigma_ss = np.zeros((self.K-1, self.K-1))
        beta_ss = np.zeros((self.A, self.K, self.V))
        bound = np.array([0] * N)
        Lambda = np.array([0] * N)
        siginv = np.linalg.inv(self.sigma)
        
        
    def __m_step__(self):
        self.stopits = True
        
    def __optimize_stm__(self, optimizer):
        if optimizer == 'em':
            # Split documents into groups
            self.groups = [[x for x in range(self.N) if x % self.ngroups == j] 
                           for j in range(self.ngroups)]
            
            # Run EM
            print "Beginning EM"
            while self.stopits == False:
                #for i in xrange(self.ngroups):
                #    gdocs = self.documents[self.groups[i]]
                #    self.__e_step__(gdocs)
                #    if self.verbose:
                #        print "Completed Group " + str(i) + " E-Step"
                self.__e_step__(documents)
                self.__m_step__()
    
    def __theta_posterior_draw__(self):
        # TODO
        return None
    
    def __compute_lhood_bound__(self):
        # TODO
        return None
    
    def __init_random__(self):
        self.mu = np.zeros((self.K-1, 1))
        self.sigma = 20 * np.identity(self.K-1)
        self.beta = np.random.gamma(shape=0.1, size=(self.A, self.K, self.V))
        self.Lambda = np.zeros((self.N, self.K-1))
        
    def __kappa_init__(self):
        
        # Baseline log probabilities
        m = np.array(self.wcounts)[0].astype(float)/ np.sum(np.array(self.wcounts)[0])
        m = np.log(m) - np.log(m)
        self.kappa['m'] = m; del m;
        
        # Parameter objects
        self.aspectmod = self.A > 1
        
        # Covariates
        self.kappa['covar'] = None
        
        
    def print_topics(self):
        print self.groups
    
    def estimate_effect(self):
        # TODO
        self.N = 30
        return None
    
    def train(self, documents, vocab=None, K=10, prevalence=None, content=None, data=None,
              max_em_its=500, init_type="random", optimizer="em", 
              em_tol=1e-5, verbose=True, report_every=5, LDAbeta=True,
              interactions=True, ngroups=10, model=None):
        """Train a STM model.
        """
        # Input parsing
        if type(documents) == sp.sparse.coo_matrix:
            self.documents = documents.tolil()
            #self.documents = documents
            
        # Parallelize the collection
        #self.dist_docs = CoordinateMatrix(sc.parallelize([MatrixEntry(i, j, x) for i, j, x 
        #                    in zip(documents.row, documents.col, documents.data)]))
        #print self.dist_docs.numCols(), self.dist_docs.numRows()
        #self.dist_
        
        # Basic setup
        self.N, self.V = documents.shape
        self.K = K
        self.verbose = verbose
        self.report_every = report_every
        self.ngroups = ngroups
        self.max_em_its = max_em_its
        self.em_tol = em_tol
        self.gamma = {
            'mode' : 'L1',
            'prior' : None,
            'enet' : 1
        }
        self.kappa = {
            'LDAbeta' : LDAbeta,
            'interactions' : interactions,
            'fixedintercept' : True,
            'mstep' : {
                'tol' : .001,
                'maxit' : 3
            },
            'contrasts' : False
        }
        self.tau = {
            'mode' : 'L1',
            'nits' : 50,
            'burnin' : 25,
            'alpha' : 50.0 / self.K,
            'eta' : .01,
            's' : .05,
            'p' : 3000,
            'd_groups_size' : 2000
        }
        self.wcounts = documents.sum(0)
        self.ntokens = np.sum(self.wcounts)
        self.stopits = False
        self.A = len(content)
        self.P = len(prevalence)
        self.X = data[prevalence]
        self.Y = data[content]
                
        # Initialization
        print "Beginning Initialization"
        if init_type == 'random':
            self.__init_random__()
        self.__kappa_init__()
        
        # Run EM
        self.__optimize_stm__(optimizer)
        
        # Declare completion
        print "All done!"

In [ ]:
stm = STM(sc)
stm.train(documents, prevalence=['Species'], content=['Species'], data=iris, ngroups=10)

In [ ]:
documents_lil = documents.tolil()

In [ ]:
sc.parallelize(documents_lil.todense()).map(hi)

In [ ]:
[(i, j, x) for i, j, x in zip(documents.row, documents.col, documents.data)]

In [ ]:
from scipy import stats
myvec = sc.parallelize([[1,2,3], [4,5,6,7], [8,9]])
myvec.map(do_something).collect()

In [ ]:
def do_something(x):
    return str(sp.stats.hmean(x)) + " hello"

## Scraps

In [ ]:
# Step 1: Initalize Parameters
#ngroups = int(settings['ngroups'][0])
#if fit['model'] == rpy2.rinterface.NULL:
#    print "Beginning Initialization"
#    model = robjects.globalenv['stm.init'](fit['documents'], fit['settings'])
#    if K == 0:
        
#else:
#    print "Restarting Model"

In [ ]:
#settings = dict(zip( fit['settings'].names, 
#         list(fit['settings'])))

In [ ]:
%%R
# Save the objects for debugging
#save(documents, betaindex, beta, lambda, mu, sigma, verbose, file="estep_prep.RData")

In [ ]:
exp_eta = np.exp(doc_item['init'])
eta = doc_item['init']
betas = doc_item['beta_i']
doc_ct = doc_item['doc'][1]
ndoc = np.sum(doc_ct)
siginv = np.linalg.inv(sigma)
part1 = np.dot(np.log(np.dot(exp_eta, betas)), doc_ct) - ndoc * np.log(np.sum(exp_eta))
diff = mu.T - eta[:K-1]
part2 = 0.5 * float(np.dot(np.dot(diff, siginv), diff.T))
#ndoc = np.sum(doc_ct)

In [ ]:
doc_item = collection[1]
doc_ct = doc_item['doc'][1]
ndoc = np.sum(doc_ct)
ndoc

In [ ]:
sigmaentropy = np.log(np.abs(np.linalg.det(sigma))) * 0.5
hi = hpb(eta=optim_par, beta=doc_item['beta_i'], doc_ct=doc_item['doc'][1],
           mu=mu, siginv=siginv, sigmaentropy=sigmaentropy)

siginv = np.linalg.inv(sigma)
likelihood(eta=doc_item['init'], beta=doc_item['beta_i'], doc_ct=doc_item['doc'][1],
           mu=mu, siginv=siginv)

grad(eta=doc_item['init'], beta=doc_item['beta_i'], doc_ct=doc_item['doc'][1],
           mu=mu, siginv=siginv)

eps = [1e-8] * 19
numerical_gradient = optimize.approx_fprime(doc_item['init'], likelihood, 
                                            eps, doc_item['beta_i'], doc_item['doc'][1], mu, siginv)
numerical_gradient

# Try optimization
eta=doc_item['init']
beta=doc_item['beta_i']
doc_ct=doc_item['doc'][1]
mu=mu
siginv = np.linalg.inv(sigma)

test = sp.optimize.minimize(likelihood, eta, args=(beta, doc_ct, mu, siginv), 
                            method='BFGS')
optim_par = test.x

doc_item.keys()

In [ ]:
# Input the data
data = sc.textFile("sample_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

In [ ]:
# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize()) + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

In [ ]:
# Sample docs
documents = sp.io.mmread("sample_docs_sparse.obj")
N, V = documents.shape
print N, V

In [ ]:
# Need to figure out prevalence
robjects.r('''
    fit = stm(processed_corpus$documents, 
             processed_corpus$vocab, 
             K=20, prevalence=~s(date), 
             data=processed_corpus$meta,
             init.type = 'Spectral',
             seed=02138)
''');

In [ ]:
def hpb(eta, beta, doc_ct, mu, siginv, sigmaentropy):
        
    # Compute the Hessian
    exp_eta = np.exp(np.append(eta, [0]))
    theta = np.reshape(exp_eta/np.sum(exp_eta), (len(exp_eta), -1)).T
    EB = np.apply_along_axis(lambda x: x * exp_eta, 0, beta)
    EB = np.apply_along_axis(lambda x: x * (np.sqrt(doc_ct).T) / np.sum(EB,0), 1, EB)
    hess = np.dot(EB, EB.T) - np.sum(doc_ct) * np.dot(theta.T, theta)    
    EB = np.apply_along_axis(lambda x: x * np.sqrt(doc_ct).T, 1, EB)
    hess[np.diag_indices_from(hess)] = hess[np.diag_indices_from(hess)] - np.sum(EB, 1) + np.sum(doc_ct) * theta
    hess = hess[:hess.shape[0]-1,:hess.shape[1]-1] + siginv
    
    # Invert via Cholesky decomposition
    try:
        nu = np.linalg.cholesky(hess)
    except:
        dvec = np.array(np.diag(hess))
        magnitudes = np.sum(np.abs(hess), 1) - abs(dvec)
        Km1 = len(dvec)
        for i in range(Km1):
            if dvec[i] < magnitudes[i]:
                dvec[i] = magnitudes[i]
        hess[np.diag_indices_from(hess)] = dvec
        nu = np.linalg.cholesky(hess)
    
    # Finish construction
    det_term = -np.sum(np.log(np.diag(nu)))
    nu = np.linalg.inv(np.triu(nu))
    nu = np.dot(nu, nu.T)
    diff = eta - mu.flatten()
    
    # Compute the bound
    bound = (np.dot(np.log(np.dot(theta, beta)), doc_ct) + det_term 
             - 0.5 * np.dot(diff.T, np.dot(siginv, diff)) - sigmaentropy)
    
    # Construct ouput
    out = {'phis': EB,
           'eta': {'lambda': eta, 'nu': nu},
           'bound': bound}
    return out